In [1]:
import sys
sys.path.append('../RecSysRep/')

import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

URM_all_dataframe, ICM_genre_all_dataframe, ICM_subgenre_all_dataframe, ICM_channel_all_dataframe, ICM_event_all_dataframe = ld.getDataframes()

import numpy as np
from scipy.sparse import *

In [3]:
ICM_event_all_dataframe

,ItemID,EpisodeID,Match
0,0,121781,1.0
1,1,150102,1.0
2,1,349614,1.0
3,2,9174,1.0
4,2,20833,1.0
...,...,...,...
358065,18056,169988,1.0
358066,18056,198728,1.0
358067,18057,237262,1.0
358068,18058,96320,1.0


In [4]:
ICM_event_all = ICM_event_all.tocsr()
episodesPerShow = np.ediff1d(ICM_event_all.indptr)

In [5]:
print(episodesPerShow)
# episodesPerShow.sort()
print(episodesPerShow)

[1 2 3 ... 2 1 2]
[1 2 3 ... 2 1 2]


In [6]:
uniqueEpisodes = set(episodesPerShow)
# print(uniqueEpisodes)

import collections
counter=collections.Counter(episodesPerShow)
episodeFrequency = list(counter.items())
# counter.most_common(20)
invEF = [(sub[1], sub[0]) for sub in episodeFrequency]

In [7]:
print(invEF)

[(3849, 1), (3603, 2), (2328, 3), (1465, 4), (41, 27), (15, 50), (406, 7), (17, 55), (597, 6), (34, 31), (138, 15), (92, 0), (414, 8), (8, 81), (202, 12), (79, 21), (892, 5), (322, 9), (189, 13), (43, 30), (56, 24), (1, 735), (94, 20), (8, 116), (250, 11), (13, 57), (39, 28), (27, 34), (21, 53), (124, 18), (1, 294), (93, 19), (66, 22), (1, 927), (3, 318), (1, 238), (19, 47), (5, 157), (4, 124), (36, 29), (64, 23), (31, 32), (26, 35), (177, 14), (12, 69), (289, 10), (12, 70), (24, 39), (12, 59), (66, 25), (1, 2699), (11, 74), (1, 503), (11, 79), (11, 51), (153, 16), (31, 38), (2, 91), (4, 207), (99, 17), (52, 26), (26, 41), (18, 43), (3, 137), (15, 63), (13, 119), (2, 272), (8, 84), (19, 62), (4, 176), (16, 54), (4, 162), (6, 83), (2, 133), (1, 660), (16, 56), (2, 202), (35, 33), (10, 58), (16, 68), (6, 128), (2, 233), (15, 61), (2, 170), (2, 477), (12, 76), (3, 195), (24, 37), (2, 267), (2, 397), (3, 249), (2, 385), (34, 36), (11, 73), (15, 48), (2, 253), (16, 87), (2, 180), (1, 659), 

In [8]:
reshapedEPR = episodesPerShow.reshape(-1,1)
print(reshapedEPR)
print(len(reshapedEPR))

[[1]
 [2]
 [3]
 ...
 [2]
 [1]
 [2]]
18059


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit(reshapedEPR)

#you can see the labels with:
print(kmeans.labels_)
print(len(kmeans.labels_))
print(kmeans.cluster_centers_)

In [36]:
import pandas as pd

a = pd.DataFrame(episodesPerShow, columns = ['NumEpisodes'])
a

,NumEpisodes
0,1
1,2
2,3
3,1
4,1
...,...
18054,1
18055,1
18056,2
18057,1


In [42]:
num_cluster = 5
div = pd.qcut(a.NumEpisodes, num_cluster, labels=list(range(0, num_cluster)))
div

0        0
1        1
2        2
3        0
4        0
        ..
18054    0
18055    0
18056    1
18057    0
18058    1
Name: NumEpisodes, Length: 18059, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [43]:
a["cluster"] = div

In [44]:
for i in range(0, num_cluster):
    print(a.loc[a['cluster'] == i].describe())

       NumEpisodes
count  3941.000000
mean      0.976656
std       0.151014
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
       NumEpisodes
count       3603.0
mean           2.0
std            0.0
min            2.0
25%            2.0
50%            2.0
75%            2.0
max            2.0
       NumEpisodes
count  3793.000000
mean      3.386238
std       0.486950
min       3.000000
25%       3.000000
50%       3.000000
75%       4.000000
max       4.000000
       NumEpisodes
count  3170.000000
mean      7.171609
std       1.976083
min       5.000000
25%       5.000000
50%       7.000000
75%       9.000000
max      11.000000
       NumEpisodes
count  3552.000000
mean     87.679336
std     220.377718
min      12.000000
25%      17.000000
50%      30.000000
75%      79.000000
max    5778.000000


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

import matplotlib as mpl
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False



# determine which K-Means cluster each point belongs to
cluster_id = KMeans(3).fit_predict(reshapedEPR)

# determine densities by cluster assignment and plot
fig, ax = plt.subplots()
bins = np.linspace(episodesPerShow.min(), episodesPerShow.max(), 40)
for ii in np.unique(cluster_id):
    subset = episodesPerShow[cluster_id==ii]
    ax.hist(subset, bins=bins, alpha=0.5, label=f"Cluster {ii}")
ax.legend()
plt.show()